In [6]:
base_dir= '/home/data3/whm/code/flare25/datasets/'

In [33]:
import os
import SimpleITK as sitk
import numpy as np

def relative_volume(dir, num_cls):
    # 获取目录下所有 .nii.gz 文件
    files = [f for f in os.listdir(dir) if f.endswith('.nii.gz')]
    
    # 初始化比例列表，每个前景类对应一个子列表
    proportions = [[] for _ in range(num_cls)]
    
    # 遍历每个文件
    for file in files:
        file_path = os.path.join(dir, file)
        
        # 使用 SimpleITK 读取 NIfTI 文件
        img = sitk.ReadImage(file_path)
        mask = sitk.GetArrayFromImage(img)  # 转换为 NumPy 数组，形状为 (z, y, x)
        
        # 找到前景区域的索引（掩码值 > 0）
        indices = np.where(mask > 0)
        if len(indices[0]) == 0:
            continue  # 如果没有前景区域，跳过该图像
        
        # 计算边界框的最小和最大索引
        min_z = np.min(indices[0])
        max_z = np.max(indices[0])
        min_y = np.min(indices[1])
        max_y = np.max(indices[1])
        min_x = np.min(indices[2])
        max_x = np.max(indices[2])
        
        # 裁剪掩码
        cropped_mask = mask[min_z:max_z+1, min_y:max_y+1, min_x:max_x+1]
        
        # 计算裁剪区域的总体素数
        total_voxels = cropped_mask.size
        
        # 计算每个前景类的比例
        for i in range(1, num_cls + 1):
            prop = np.sum(cropped_mask == i) / total_voxels
            proportions[i - 1].append(prop)  # 存储比例，类 i 对应索引 i-1
    
    # 计算每类比例的均值和标准差
    mean_list = [np.mean(props) for props in proportions]
    std_list = [np.std(props) for props in proportions]
    
    return mean_list, std_list

In [35]:
CT_train_lab_path="/home/data3/whm/code/flare25/datasets/CT/CT_label"

# 计算 CT 训练集的前景比例
mean_list_ct, std_list_ct = relative_volume(CT_train_lab_path, num_cls=13)
print("CT Train Mean Proportions:", mean_list)
print("CT Train Std Proportions:", std_list)


CT Train Mean Proportions: [0.1256382520337585, 0.015333825782262379, 0.01750245633838423, 0.007333140633324921, 0.007121911500786096, 0.006662379036015619, 0.00033386167861718314, 0.00040018913625063324, 0.0025801793220234844, 0.0012088452990713396, 0.026424460678714966, 0.00591822411781959, 0.015526555935504031]
CT Train Std Proportions: [0.022930708873490812, 0.003561097937928093, 0.007826006036506987, 0.002898593391740061, 0.0017434684713766128, 0.001612543408576953, 7.298711812654644e-05, 8.520402884483831e-05, 0.0016385196395271738, 0.00028924114727595546, 0.01215254699814758, 0.001413290342434358, 0.004357759671957166]


In [51]:
import os
import nibabel as nib
import numpy as np
from skimage.metrics import adapted_rand_error
from skimage.measure import label
import glob

def check_proportions(mask, mean_list, std_list, num_cls):
    """
    检查 mask 中每类前景的比例是否在 mean ± 3 * std 范围内。
    
    参数:
        mask: 输入的掩码 (numpy array)
        mean_list: 每类前景的均值列表
        std_list: 每类前景的标准差列表
        num_cls: 前景类别数
    
    返回:
        bool: 如果所有前景比例都在范围内，返回 True，否则返回 False
    """
    # 找到前景区域的索引（掩码值 > 0）
    indices = np.where(mask > 0)
    if len(indices[0]) == 0:
        return False  # 如果没有前景区域，跳过
    
    # 计算边界框的最小和最大索引
    min_z = np.min(indices[0])
    max_z = np.max(indices[0])
    min_y = np.min(indices[1])
    max_y = np.max(indices[1])
    min_x = np.min(indices[2])
    max_x = np.max(indices[2])
    
    # 裁剪掩码到前景区域
    cropped_mask = mask[min_z:max_z+1, min_y:max_y+1, min_x:max_x+1]
    
    # 计算裁剪区域的总体素数
    total_voxels = cropped_mask.size
    
    # 检查每类前景的比例
    for i in range(1, num_cls + 1):
        prop = np.sum(cropped_mask == i) / total_voxels
        mean = mean_list[i - 1]
        std = std_list[i - 1]
        if not (mean - 3 * std <= prop <= mean + 3 * std):
            return False
    return True

def calculate_dice(mask1, mask2, num_classes=13):
    """
    计算两个掩码的 Dice 系数之和。
    
    参数:
        mask1, mask2: 输入的两个掩码 (numpy array)
        num_classes: 前景类别数
    
    返回:
        float: 所有类别的 Dice 系数之和
    """
    dice_scores = []
    for i in range(1, num_classes + 1):  # 类别从 1 到 13
        # 创建前景 mask
        mask1_class = (mask1 == i).astype(np.uint8)
        mask2_class = (mask2 == i).astype(np.uint8)

        # 计算 Dice 系数
        intersection = np.sum(mask1_class * mask2_class)
        union = np.sum(mask1_class) + np.sum(mask2_class)
        if union == 0:
            dice = 1.0  # 如果两个 mask 都为空，视为完全一致
        else:
            dice = 2.0 * intersection / union
        dice_scores.append(dice)

    return np.sum(dice_scores)/num_classes  # 返回所有类别的 Dice 系数之和

def process_masks(folder1, folder2, output_txt, mean_list, std_list, num_classes=13):
    """
    处理两个文件夹中的掩码文件，计算 Dice 系数，并在前景比例不符合条件时跳过样本。
    
    参数:
        folder1, folder2: 两个掩码文件夹路径
        output_txt: 输出结果文件路径
        mean_list: 每类前景的均值列表
        std_list: 每类前景的标准差列表
        num_classes: 前景类别数，默认值为 13
    
    输出:
        将每个样本的 Dice 系数写入 output_txt 文件
    """
    # 获取文件夹1内所有文件路径
    files1 = sorted(glob.glob(os.path.join(folder1, "*.nii.gz")))
    print(f"Processing {len(files1)} files from {folder1} and {folder2}...")
    with open(output_txt, 'w') as out_file:
        for file1 in files1:
            # 获取文件1的样本名称
            case_name = os.path.basename(file1)
            file2 = os.path.join(folder2, case_name)  # 在文件夹2中寻找对应的文件

            # 如果文件夹2中没有找到对应的文件，跳过此样本
            if not os.path.exists(file2):
                print(f"Warning: {file2} not found, skipping.")
                continue

            # 读取 nii.gz 文件
            mask1 = nib.load(file1).get_fdata()
            mask2 = nib.load(file2).get_fdata()

            # 确保文件大小一致
            assert mask1.shape == mask2.shape, f"File sizes don't match: {file1}, {file2}"

            # 检查 mask 类别数量
            if len(np.unique(mask1)) != num_classes + 1 or len(np.unique(mask2)) != num_classes + 1:
                print(f"Warning: {file1} or {file2} does not have the expected number of classes, skipping.")
                continue

            # 检查前景相对比例
            if not check_proportions(mask1, mean_list, std_list, num_classes):
                print(f"Warning: {file1} foreground proportions out of range, skipping.")
                continue
            if not check_proportions(mask2, mean_list, std_list, num_classes):
                print(f"Warning: {file2} foreground proportions out of range, skipping.")
                continue

            # 计算 Dice 分数之和
            score = calculate_dice(mask1, mask2, num_classes)

            # 保存结果
            out_file.write(f"{case_name} {score}\n")
            print(f"Processed {case_name}, Score: {score}")

In [ ]:
# 用前景比例分布
# liver xxx%...; 统计mean std...
# relative volume

In [40]:
# 设置文件夹路径和输出文件, pseudo CT, official
folder1_CT = base_dir+'FLARE-Task3-DomainAdaption/train_CT_pseudolabel/flare22_aladdin5_pseudo/'
folder2_CT = base_dir+'FLARE-Task3-DomainAdaption/train_CT_pseudolabel/pseudo_label_blackbean_flare22/'
output_txt_CT = base_dir+'FLARE-Task3-DomainAdaption/train_CT_pseudolabel/output_cls_13_scores.txt'

# 执行处理
process_masks(folder1_CT, folder2_CT, output_txt_CT, mean_list_ct, std_list_ct, num_classes=13)

Processed Case_00001.nii.gz, Score: 0.9270468822794856
Processed Case_00003.nii.gz, Score: 0.9079135792936632
Processed Case_00004.nii.gz, Score: 0.9297449813429431
Processed Case_00005.nii.gz, Score: 0.880656767917057
Processed Case_00006.nii.gz, Score: 0.8658336126703686
Processed Case_00009.nii.gz, Score: 0.9465103517923306
Processed Case_00015.nii.gz, Score: 0.9251789449272773
Processed Case_00016.nii.gz, Score: 0.9339170132255382
Processed Case_00018.nii.gz, Score: 0.913021604959557
Processed Case_00023.nii.gz, Score: 0.941310146150303
Processed Case_00025.nii.gz, Score: 0.8561331237444695
Processed Case_00026.nii.gz, Score: 0.9343955551589845
Processed Case_00029.nii.gz, Score: 0.9342248720044312
Processed Case_00033.nii.gz, Score: 0.9416345031216561
Processed Case_00036.nii.gz, Score: 0.9339728473475831
Processed Case_00038.nii.gz, Score: 0.8340019509215768
Processed Case_00039.nii.gz, Score: 0.869651938896948
Processed Case_00042.nii.gz, Score: 0.9169628798679751
Processed Case

In [41]:
# 设置文件夹路径和输出文件 amos, ours
# /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/CT_MR_TJU_init2
folder1_amos = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/CT_MR_TJU_init2'
folder2_amos = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/CT_MR_TJU_init3'
output_txt_amos = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/output_cls_13_scores.txt'

# 执行处理
process_masks(folder1_amos, folder2_amos, output_txt_amos, mean_list_ct, std_list_ct, num_classes=13)

Processed amos_7014.nii.gz, Score: 0.9537088310908023
Processed amos_7017.nii.gz, Score: 0.9302501294329221
Processed amos_7018.nii.gz, Score: 0.9043948253835162
Processed amos_7030.nii.gz, Score: 0.7740930325100628
Processed amos_7031.nii.gz, Score: 0.9215822361380623
Processed amos_7035.nii.gz, Score: 0.8396969530330829
Processed amos_7075.nii.gz, Score: 0.7734774109497555
Processed amos_7085.nii.gz, Score: 0.8189407089168224
Processed amos_7086.nii.gz, Score: 0.8787978371811666
Processed amos_7087.nii.gz, Score: 0.8701861983568754
Processed amos_7088.nii.gz, Score: 0.9223743165880987
Processed amos_7098.nii.gz, Score: 0.9403287273722796
Processed amos_7112.nii.gz, Score: 0.9264011660226129
Processed amos_7121.nii.gz, Score: 0.8879991066153123
Processed amos_7122.nii.gz, Score: 0.896286034035888
Processed amos_7123.nii.gz, Score: 0.8876125499005492
Processed amos_7125.nii.gz, Score: 0.8228175259888938
Processed amos_7126.nii.gz, Score: 0.8310640953416327
Processed amos_7134.nii.gz, S

In [42]:
# 设置文件夹路径和输出文件 LLD, ours
# /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/CT_MR_TJU_init2
folder1_LLD = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/CT_MR_TJU_init2'
folder2_LLD = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/CT_MR_TJU_init3'
output_txt_LLD = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/output_cls_13_scores.txt'

# 执行处理
process_masks(folder1_LLD, folder2_LLD, output_txt_LLD, mean_list_ct, std_list_ct, num_classes=13)

Processed MR-391135_1_C+A.nii.gz, Score: 0.9382087749695446
Processed MR-391135_1_C+Delay.nii.gz, Score: 0.9465551238488611
Processed MR-391135_1_C+V.nii.gz, Score: 0.9109443146316779
Processed MR-391135_1_C-pre.nii.gz, Score: 0.9497764944125762
Processed MR-391135_1_DWI.nii.gz, Score: 0.9033165190846041
Processed MR-391135_1_InPhase.nii.gz, Score: 0.8604226217620609
Processed MR-391135_1_T2WI.nii.gz, Score: 0.91902003463966
Processed MR-398189_1_C+A.nii.gz, Score: 0.9282887774547075
Processed MR-398189_1_C+Delay.nii.gz, Score: 0.9526978476023424
Processed MR-398189_1_C+V.nii.gz, Score: 0.9470627775994607
Processed MR-398189_1_C-pre.nii.gz, Score: 0.9455768380891352
Processed MR-398189_1_InPhase.nii.gz, Score: 0.8394239071840027
Processed MR-398189_1_OutPhase.nii.gz, Score: 0.8663834685165184
Processed MR-398189_1_T2WI.nii.gz, Score: 0.909376615350015
Processed MR-398374_2_C+A.nii.gz, Score: 0.9439173738184249
Processed MR-398374_2_C+Delay.nii.gz, Score: 0.9441537051573599
Processed MR

In [45]:
# 设置文件夹路径和输出文件 PET, 13, ours
# /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli
folder1_PET13 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/_13'
folder2_PET13 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_init3/_13'
output_txt_PET13 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/output_cls_13_scores.txt'

# 执行处理
process_masks(folder1_PET13, folder2_PET13, output_txt_PET13, mean_list_ct, std_list_ct, num_classes=13)

Processed fdg_04606080a0_02-20-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-22538.nii.gz, Score: 0.7083164053604788
Processed fdg_0fa313309d_04-23-2005-NA-Unspecified_CT-29055.nii.gz, Score: 0.8278144732371209
Processed fdg_1781ba966c_08-08-2002-NA-PET-CT_Ganzkoerper__primaer_mit_KM-99061.nii.gz, Score: 0.8076905767898277
Processed fdg_1a90052cb2_09-06-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-80674.nii.gz, Score: 0.8408056010339784
Processed fdg_1fa22c576e_10-17-2004-NA-PET-CT_Ganzkoerper__primaer_mit_KM-16838.nii.gz, Score: 0.7991575805299541
Processed fdg_245182006a_11-04-2000-NA-PET-CT_Ganzkoerper__primaer_mit_KM-02942.nii.gz, Score: 0.8161914989012851
Processed fdg_274688d0a8_03-24-2007-NA-PET-CT_Ganzkoerper__primaer_mit_KM-59557.nii.gz, Score: 0.6909408161839891
Processed fdg_2a78eed085_12-22-2001-NA-PET-CT_Ganzkoerper__primaer_mit_KM-17302.nii.gz, Score: 0.7413137623147611
Processed fdg_321bba14bc_03-07-2002-NA-PET-CT_Ganzkoerper__primaer_mit_KM-18724.nii.gz, Score: 0.82653184479

In [44]:
# 设置文件夹路径和输出文件 PET, 4, ours
# /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/pet_train_pet2_1
# /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/pet_train_pet_ct2_2
folder1_PET4 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/pet_train_pet2_1'
folder2_PET4 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/pet_train_pet_ct2_2'
output_txt_PET4 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/output_cls_4_scores.txt'
mean_list_pet =  mean_list_ct[:3]+[mean_list_ct[12]]
std_list_pet =  std_list_ct[:3]+[std_list_ct[12]]
# 执行处理
process_masks(folder1_PET4, folder2_PET4, output_txt_PET4, mean_list_pet, std_list_pet,num_classes=4)

Processed fdg_04606080a0_02-20-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-22538.nii.gz, Score: 0.9683135633983581
Processed fdg_05808cf24e_03-18-2006-NA-PET-CT_Ganzkoerper__primaer_mit_KM-91344.nii.gz, Score: 0.8592855721286479
Processed fdg_08cdb15e0b_11-21-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-47234.nii.gz, Score: 0.4114472492415361
Processed fdg_0fa313309d_04-23-2005-NA-Unspecified_CT-29055.nii.gz, Score: 0.9893502242855268
Processed fdg_11e258cc1f_03-06-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-87979.nii.gz, Score: 0.966157662550015
Processed fdg_1781ba966c_08-08-2002-NA-PET-CT_Ganzkoerper__primaer_mit_KM-99061.nii.gz, Score: 0.981377954338843
Processed fdg_182bdeba22_05-10-2007-NA-PET-CT_Ganzkoerper__primaer_mit_KM-88669.nii.gz, Score: 0.7290886371530878
Processed fdg_185da4c8b6_04-18-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-06919.nii.gz, Score: 0.9898491336128908
Processed fdg_1a3d4e63ee_07-03-2004-NA-PET-CT_Ganzkoerper__primaer_mit_KM-34298.nii.gz, Score: 0.9715041038453

In [62]:
# 设置文件夹路径和输出文件 PET, 4, ours

folder1_PET13_4 = base_dir+r'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_init3/o_4/'
folder2_PET13_4 = base_dir+r'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/o_4/'

output_txt_PET13_4 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/output_cls_13_4_scores.txt'
mean_list_pet =  mean_list_ct[:3]+[mean_list_ct[12]]
std_list_pet =  std_list_ct[:3]+[std_list_ct[12]]
print("mean_list_pet:", mean_list_pet)
print("std_list_pet:", std_list_pet)
print("folder2_PET13_4:", folder2_PET13_4)

# 执行处理
process_masks(folder1_PET13_4, folder2_PET13_4, output_txt_PET13_4, mean_list_pet, std_list_pet,num_classes=4)

mean_list_pet: [0.1256382520337585, 0.015333825782262379, 0.01750245633838423, 0.015526555935504031]
std_list_pet: [0.022930708873490812, 0.003561097937928093, 0.007826006036506987, 0.004357759671957166]
folder2_PET13_4: /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/o_4/
Processing 1000 files from /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_init3/o_4/ and /home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/o_4/...
Processed fdg_04606080a0_02-20-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-22538.nii.gz, Score: 0.8747051295621989
Processed fdg_0fa313309d_04-23-2005-NA-Unspecified_CT-29055.nii.gz, Score: 0.9593301051325244
Processed fdg_11e258cc1f_03-06-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-87979.nii.gz, Score: 0.9118493241178237
Processed fdg_1781ba966c_08-08-2002-NA-PET-CT_Ganzkoerper__primaer_mit_KM-99061.nii.gz, Score: 0.9471373723828334
Pro

In [ ]:
# 过滤标签： 
# 3.过滤样本    (1)4类， 13类 ； (2) top2模型相似性
# Samples: 
#   pseudo CT: 2000 -> ~1500； 排序选择 top 200?
#   MR-amos: 800 -> ~500; top 200?
#   MR-LLD: 4000 -> ~3700; top200* 8sequence
#   PET-4: 1000 -> ~950
#   PET-13: 1000 -> ~500+

In [ ]:
# 过滤标签： 
# 3.过滤样本    (1)4类， 13类 ； (2) relative volume of foregrounds; (3) top2模型相似性 
# Samples: 
#   pseudo CT: 2000 -> ~900； 排序选择 top 200?
#   MR-amos: 800 -> ~260; top 200?
#   MR-LLD: 4000 -> ~2200; top200* 8 sequence
#   PET-4: 1000 -> ~860
#   PET-13: 1000 -> ~140 ; pet 全部13类，某些前景分割结果不理想
#   PET-13-4: 1000 -> ~ 800; 

In [ ]:
modas = ['amos','OutPhase', 'C+Delay', 'C-pre', 'C+V', 'C+A', 'InPhase', 'DWI', 'T2WI', 'fdg','psma']

In [75]:
def select_samples_topk(input_txt, output_txt, topk=200, keywords=modas, score_bar=0.9):
    # Read the input file
    with open(input_txt, 'r') as file:
        lines = file.readlines()

    # Initialize dictionary to store samples for each modality
    modality_samples = {modality: [] for modality in keywords}

    # Group samples by modality
    for line in lines:
        sample_name, score = line.split()
        if float(score) < score_bar:
            continue
        for modality in keywords:
            if modality in sample_name:
                modality_samples[modality].append((sample_name, float(score)))
                break

    # Sort samples by score for each modality and select top-k
    selected_samples = []
    for modality in keywords:
        if modality_samples[modality]:  # If there are samples for this modality
            # Sort by score in descending order and select top-k
            sorted_modality = sorted(modality_samples[modality], key=lambda x: x[1], reverse=True)
            topk = min(topk, len(sorted_modality))  # Ensure we don't exceed available samples
            print(f"Selected top {topk} samples for modality {modality}:")
            for sample in sorted_modality[:topk]:
                print(f"Selected {sample[0]} with score {sample[1]} for modality {modality}")
              
            selected_samples.extend([sample[0] for sample in sorted_modality[:topk]])

    # Save selected sample names to output file
    with open(output_txt, 'w') as out_file:
        for sample_name in selected_samples:
            out_file.write(sample_name + '\n')

    print(f"Selected samples saved to {output_txt}")

In [ ]:
# 设置文件夹路径和输出文件, pseudo CT, official
modas = ['Case_']
output_txt_CT = base_dir+'FLARE-Task3-DomainAdaption/train_CT_pseudolabel/output_cls_13_scores.txt'
selected_output_txt_CT = base_dir+'FLARE-Task3-DomainAdaption/train_CT_pseudolabel/output_cls_13_scores_topk.txt'
select_samples_topk(output_txt_CT, selected_output_txt_CT, topk=100, keywords=modas, score_bar=0.9)

Selected Case_00055.nii.gz with score 0.9537045870919628 for modality Case_
Selected Case_01118.nii.gz with score 0.952226969811186 for modality Case_
Selected Case_00061.nii.gz with score 0.9519402711235345 for modality Case_
Selected Case_01089.nii.gz with score 0.9516642215954169 for modality Case_
Selected Case_00679.nii.gz with score 0.951058247528298 for modality Case_
Selected Case_00200.nii.gz with score 0.9503929486492466 for modality Case_
Selected Case_00937.nii.gz with score 0.9502839106443334 for modality Case_
Selected Case_01425.nii.gz with score 0.9501353197557066 for modality Case_
Selected Case_00433.nii.gz with score 0.9495968885018777 for modality Case_
Selected Case_00704.nii.gz with score 0.9493256194827192 for modality Case_
Selected Case_00186.nii.gz with score 0.9488198102223874 for modality Case_
Selected Case_00412.nii.gz with score 0.9487640345763088 for modality Case_
Selected Case_00613.nii.gz with score 0.9486945018011761 for modality Case_
Selected Case_

In [69]:
modas = ['amos','OutPhase', 'C+Delay', 'C-pre', 'C+V', 'C+A', 'InPhase', 'DWI', 'T2WI', 'fdg','psma']

In [72]:
# 设置文件夹路径和输出文件 amos, ours
output_txt_amos = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/output_cls_13_scores.txt'
output_txt_amos_topk = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/output_cls_13_scores_topk.txt'
select_samples_topk(output_txt_amos, output_txt_amos_topk, topk=100, keywords=modas, score_bar=0.9)


Selected top 100 samples for modality amos:
Selected amos_7749.nii.gz with score 0.9673538852019963 for modality amos
Selected amos_7933.nii.gz with score 0.9656591839185213 for modality amos
Selected amos_8099.nii.gz with score 0.9650412867948565 for modality amos
Selected amos_7839.nii.gz with score 0.9647777173368013 for modality amos
Selected amos_7778.nii.gz with score 0.9641843953090325 for modality amos
Selected amos_7750.nii.gz with score 0.963424576769506 for modality amos
Selected amos_7889.nii.gz with score 0.9631400662265779 for modality amos
Selected amos_7932.nii.gz with score 0.9622421567746339 for modality amos
Selected amos_7632.nii.gz with score 0.961290389539192 for modality amos
Selected amos_7325.nii.gz with score 0.9611520094146383 for modality amos
Selected amos_7934.nii.gz with score 0.9608276633759179 for modality amos
Selected amos_8071.nii.gz with score 0.960472483509439 for modality amos
Selected amos_7245.nii.gz with score 0.9601588392841846 for modality am

In [73]:
output_txt_LLD = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/output_cls_13_scores.txt'
output_txt_LLD_topk = base_dir+'pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/output_cls_13_scores_topk.txt'

# 执行处理
select_samples_topk(output_txt_LLD, output_txt_LLD_topk, topk=100, keywords=modas, score_bar=0.9)

Selected top 100 samples for modality OutPhase:
Selected MR-459946_5_OutPhase.nii.gz with score 0.9427212505204168 for modality OutPhase
Selected MR182036_0_OutPhase.nii.gz with score 0.9405517334663782 for modality OutPhase
Selected MR69723_5_OutPhase.nii.gz with score 0.9325723679862352 for modality OutPhase
Selected MR-451771_5_OutPhase.nii.gz with score 0.9321194504962964 for modality OutPhase
Selected MR78054_1_OutPhase.nii.gz with score 0.9316591565776786 for modality OutPhase
Selected MR181910_0_OutPhase.nii.gz with score 0.9315202336856946 for modality OutPhase
Selected MR180939_0_OutPhase.nii.gz with score 0.9314651922993359 for modality OutPhase
Selected MR201576_0_OutPhase.nii.gz with score 0.9313680800018158 for modality OutPhase
Selected MR177162_0_OutPhase.nii.gz with score 0.9293875818747532 for modality OutPhase
Selected MR58748_5_OutPhase.nii.gz with score 0.9289806265878481 for modality OutPhase
Selected MR189354_5_OutPhase.nii.gz with score 0.9289290453140222 for mod

In [78]:
output_txt_PET13 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/output_cls_13_scores.txt'
output_txt_PET13_topk = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/CT_MR_TJU_nonli/output_cls_13_scores_topk.txt'

# 执行处理
select_samples_topk(output_txt_PET13, output_txt_PET13_topk, topk=100, keywords=modas, score_bar=0.82)

Selected top 14 samples for modality fdg:
Selected fdg_556b56485a_08-29-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-83019.nii.gz with score 0.873611106921525 for modality fdg
Selected fdg_bd7603e9df_05-06-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-26867.nii.gz with score 0.8620604143754633 for modality fdg
Selected fdg_e6f6c84ec5_11-05-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-82117.nii.gz with score 0.8538833571617856 for modality fdg
Selected fdg_f15b868db6_02-03-2006-NA-PET-CT_Ganzkoerper__primaer_mit_KM-17521.nii.gz with score 0.8493941356107567 for modality fdg
Selected fdg_9f7c68f5ca_12-21-2001-NA-PET-CT_Ganzkoerper__primaer_mit_KM-99932.nii.gz with score 0.8423264462924054 for modality fdg
Selected fdg_1a90052cb2_09-06-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-80674.nii.gz with score 0.8408056010339784 for modality fdg
Selected fdg_80b8393c44_07-22-2002-NA-PET-CT_Teilkoerper__primaer_mit_KM-92543.nii.gz with score 0.8321824604164321 for modality fdg
Selected fdg_ccb9c375b2_10-0

In [83]:
output_txt_PET4 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/output_cls_4_scores.txt'
output_txt_PET4_topk = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/output_cls_4_scores_topk.txt'

# 执行处理
select_samples_topk(output_txt_PET4, output_txt_PET4_topk, topk=100, keywords=modas, score_bar=0.9)

Selected top 100 samples for modality fdg:
Selected fdg_85b69fed1c_01-21-2006-NA-PET-CT_Ganzkoerper__primaer_mit_KM-08512.nii.gz with score 0.9909627246134851 for modality fdg
Selected fdg_185da4c8b6_04-18-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-06919.nii.gz with score 0.9898491336128908 for modality fdg
Selected fdg_9f6e8b1b43_11-16-2002-NA-PET-CT_Ganzkoerper_nativ-64835.nii.gz with score 0.989555211974321 for modality fdg
Selected fdg_0fa313309d_04-23-2005-NA-Unspecified_CT-29055.nii.gz with score 0.9893502242855268 for modality fdg
Selected fdg_556b56485a_08-29-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-83019.nii.gz with score 0.988667685677981 for modality fdg
Selected fdg_b6fc20942c_12-28-2001-NA-PET-CT_Ganzkoerper__primaer_mit_KM-65823.nii.gz with score 0.9884587604780427 for modality fdg
Selected fdg_bd7603e9df_05-06-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-26867.nii.gz with score 0.9883222730414256 for modality fdg
Selected fdg_c73c2685c0_02-01-2001-NA-PET-CT_Ganzkoerper__

In [81]:
output_txt_PET_13_4 = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/output_cls_13_4_scores.txt'
output_txt_PET13_4_topk = base_dir+'pseudolab/1adj_spacing/train_PET_unlabeled/output_cls_13_4_scores_topk.txt'

# 执行处理
select_samples_topk(output_txt_PET_13_4, output_txt_PET13_4_topk, topk=100, keywords=modas, score_bar=0.9)

Selected top 100 samples for modality fdg:
Selected fdg_185da4c8b6_04-18-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-06919.nii.gz with score 0.9690770648530513 for modality fdg
Selected fdg_9f6e8b1b43_11-16-2002-NA-PET-CT_Ganzkoerper_nativ-64835.nii.gz with score 0.965701924892471 for modality fdg
Selected fdg_9f7c68f5ca_12-21-2001-NA-PET-CT_Ganzkoerper__primaer_mit_KM-99932.nii.gz with score 0.9639038408360001 for modality fdg
Selected fdg_c787334539_01-05-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-93319.nii.gz with score 0.9627955047192194 for modality fdg
Selected fdg_85b69fed1c_01-21-2006-NA-PET-CT_Ganzkoerper__primaer_mit_KM-08512.nii.gz with score 0.9623155136691617 for modality fdg
Selected fdg_9ffd8409b3_07-15-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-58670.nii.gz with score 0.9617263608742964 for modality fdg
Selected fdg_f47e31ceb5_07-26-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-36586.nii.gz with score 0.9616520056623781 for modality fdg
Selected fdg_556b56485a_08-29-2003-NA

In [11]:
# 得到过滤的样本
# 过滤标签：
# 4. 对粗筛样本，过滤前景： (1) 多模型投票； (2) 前景像素的个数？

In [5]:
import os
import SimpleITK as sitk
import numpy as np
from glob import glob
import argparse
from os.path import join as ospj

def lab_voter(txt_file, esb_dir: list, out_dir, threhold=0.5):
    os.makedirs(out_dir, exist_ok=True)

    # 读取选中的样本文件名
    with open(txt_file, 'r') as f:
        selected_samples = [line.strip() for line in f.readlines()]

    # 获取所有预测文件名
    # pred_files = sorted(glob(os.path.join(esb_dir[0], '*.nii.gz')))
    pred_files = sorted([ospj(esb_dir[0], sample) for sample in selected_samples])

    for pred_file in pred_files:
        filename = os.path.basename(pred_file)
        predictions = []

        for model_dir in esb_dir:
            model_pred_path = os.path.join(model_dir, filename)
            pred_img = sitk.ReadImage(model_pred_path)
            pred_array = sitk.GetArrayFromImage(pred_img)
            predictions.append(pred_array)

        # 转换为numpy数组，形状：(n_models, D, H, W)
        predictions_np = np.stack(predictions, axis=0)

        # 获取所有可能的标签值（包含所有模型中的所有标签）
        unique_labels = np.unique(predictions_np)
        unique_labels = unique_labels[unique_labels != 0]  # 忽略背景0

        # 初始化输出标签图像
        label_vote = np.zeros_like(predictions_np[0], dtype=np.uint8)

        for label in unique_labels:
            # 对每个类别进行投票计数
            label_votes = np.sum(predictions_np == label, axis=0)

            # 创建当前类别满足共识阈值的掩码
            label_mask = label_votes >= (len(esb_dir) * threhold)

            # 只在未标注的位置填充，防止多类冲突
            conflict_mask = (label_mask & (label_vote == 0))
            label_vote[conflict_mask] = label

        # 保存结果图像
        result_img = sitk.GetImageFromArray(label_vote)
        result_img.CopyInformation(pred_img)
        sitk.WriteImage(result_img, os.path.join(out_dir, filename))

In [ ]:
# amos 投票
esb_ckpt_base = "/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/"
Esb_dir=[
'CT_MR_TJU_init3',
'CT_MR_TJU_init2',
'CT_MR_TJU_WJH',
'CT550',
'CT_MR_TJU_nonli',
'CT_MR_TJU_init1',
]
Esb_dir = [esb_ckpt_base+ str(it) for it in Esb_dir]
Out_dir = esb_ckpt_base + 'voted_lab'

select_sample_names = esb_ckpt_base+'output_cls_13_scores_topk.txt' # TOP 100
lab_voter(select_sample_names, esb_dir=Esb_dir, out_dir=Out_dir)

In [ ]:
# LLD 投票
esb_ckpt_base = "/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/"
Esb_dir=[
'CT_MR_TJU_init3',
'CT_MR_TJU_init2',
'CT_MR_TJU_WJH',
'CT550',
'CT_MR_TJU_nonli',
'CT_MR_TJU_init1',
]
Esb_dir = [esb_ckpt_base+ str(it) for it in Esb_dir]
Out_dir = esb_ckpt_base + 'voted_lab'

select_sample_names = esb_ckpt_base+'output_cls_13_scores_topk.txt' # TOP 100
lab_voter(select_sample_names, esb_dir=Esb_dir, out_dir=Out_dir)

In [ ]:
import os
import shutil

def copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder):
    # 读取选中的样本文件名
    with open(txt_file, 'r') as f:
        selected_samples = [line.strip() for line in f.readlines()]

    # 确保目标文件夹存在，如果不存在则创建
    os.makedirs(new_image_folder, exist_ok=True)
    os.makedirs(new_label_folder, exist_ok=True)

    # 遍历图像文件夹，复制选中的文件
    for sample_name in selected_samples:
        image_file = os.path.join(image_folder, sample_name.replace('.nii', '_0000.nii'))
        label_file = os.path.join(label_folder, sample_name)

        # 如果图像和标签文件都存在，则复制到新的文件夹
        # print(image_file)
        # print(label_file)
        if os.path.exists(image_file) and os.path.exists(label_file):
            # 复制图像文件
            shutil.copy(image_file, new_image_folder)
            # 复制标签文件
            shutil.copy(label_file, new_label_folder)
            print(f"Copied {sample_name} to new folders.")
        else:
            print(f"Warning: {sample_name} not found in either image or label folder.")

In [ ]:
# pseudo CT copy top 100 to new folder
# 设置路径
ct_base_dir = "/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/"
txt_file = ct_base_dir + 'output_cls_13_scores_topk.txt'   # 包含选中样本名的txt文件
image_folder = ct_base_dir + 'imagesTr'  # 图像文件夹路径
label_folder = ct_base_dir + 'flare22_aladdin5_pseudo'  # 标签文件夹路径
new_image_folder = ct_base_dir + 'imageTr_top100'  # 新图像文件夹路径
new_label_folder = ct_base_dir + 'flare22_aladdin5_pseudo_top100'  # 新标签文件夹路径

# 调用函数
copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder)

/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_00055_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/flare22_aladdin5_pseudo/Case_00055.nii.gz
Copied Case_00055.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_01118_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/flare22_aladdin5_pseudo/Case_01118.nii.gz
Copied Case_01118.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_00061_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/flare22_aladdin5_pseudo/Case_00061.nii.gz
Copied Case_00061.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_01089_0000.nii.gz
/home/data3/whm/code/

In [ ]:
# pseudo CT 投票
esb_ckpt_base = "/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/"
Esb_dir=[
'flare22_aladdin5_pseudo',
'pseudo_label_blackbean_flare22',
'labelTr_top100/CT_MR_TJU_init3',
'labelTr_top100/CT_MR_TJU_init2',
'labelTr_top100/CT_MR_TJU_WJH',
'labelTr_top100/CT550',
'labelTr_top100/CT_MR_TJU_nonli',
'labelTr_top100/CT_MR_TJU_init1',
]
Esb_dir = [esb_ckpt_base+ str(it) for it in Esb_dir]
Out_dir = esb_ckpt_base + 'voted_lab'

select_sample_names = esb_ckpt_base+'output_cls_13_scores_topk.txt' # TOP 100
lab_voter(select_sample_names, esb_dir=Esb_dir, out_dir=Out_dir)

In [ ]:
# PET - 13 投票
esb_ckpt_base = "/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/"
Esb_dir=[
'CT_MR_TJU_nonli/_13',
'CT_MR_TJU_init3/_13',
'CT550/_13',
'CT_MR_TJU_init2/_13',
'CT_MR_TJU_WJH/_13',
]
Esb_dir = [esb_ckpt_base+ str(it) for it in Esb_dir]
Out_dir = esb_ckpt_base + 'voted_lab/o_13'

select_sample_names = esb_ckpt_base+'CT_MR_TJU_nonli/output_cls_13_scores_topk.txt'
lab_voter(select_sample_names, esb_dir=Esb_dir, out_dir=Out_dir)

In [ ]:
# PET - 4 投票
esb_ckpt_base = "/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/"
Esb_dir=[
'CT_MR_TJU_nonli/o_4',
'CT_MR_TJU_init3/o_4',
'CT550/o_4',
'pet_train_pet2_1',
'pet_train_pet_ct2_2',
'pet_train_pet_1',
'pet_train_pet2_1_1',
]
Esb_dir = [esb_ckpt_base+ str(it) for it in Esb_dir]
Out_dir = esb_ckpt_base + 'voted_lab/o_4'

select_sample_names = esb_ckpt_base+'output_cls_13_4_scores_topk.txt'
lab_voter(select_sample_names, esb_dir=Esb_dir, out_dir=Out_dir)

In [12]:
# copy selected voted top 100 CT samples to new folder
ct_base_dir = "/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/"
txt_file = ct_base_dir + 'output_cls_13_scores_topk.txt'   # 包含选中样本名的txt文件
image_folder = ct_base_dir + 'imagesTr'  # 图像文件夹路径
label_folder = ct_base_dir + 'voted_lab'  # 标签文件夹路径

new_folder_base = "/home/data3/whm/code/flare25/datasets/voted_sample/train_CT_pseudolabel/"
new_image_folder = new_folder_base + 'imageTr_top100'  # 新图像文件夹路径
new_label_folder = new_folder_base + 'voted_lab_top100'  # 新标签文件夹路径

# 调用函数
copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder)

/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_00055_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/voted_lab/Case_00055.nii.gz
Copied Case_00055.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_01118_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/voted_lab/Case_01118.nii.gz
Copied Case_01118.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_00061_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/voted_lab/Case_00061.nii.gz
Copied Case_00061.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/train_CT_pseudolabel/imagesTr/Case_01089_0000.nii.gz
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaptio

In [13]:
# copy selected voted amos samples to new folder
base_dir = "/home/data3/whm/code/flare25/datasets/"
dataset='AMOS-833'

image_folder = base_dir + f'FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/{dataset}/'  # 图像文件夹路径
label_folder = base_dir + f'pseudolab/1adj_spacing/train_MRI_unlabeled/{dataset}/voted_lab'  # 标签文件夹路径
txt_file = base_dir + f'pseudolab/1adj_spacing/train_MRI_unlabeled/{dataset}/output_cls_13_scores_topk.txt'   # 包含选中样本名的txt文件

new_folder_base = f"/home/data3/whm/code/flare25/datasets/voted_sample/{dataset}/"
new_image_folder = new_folder_base + 'imageTr_top100'  # 新图像文件夹路径
new_label_folder = new_folder_base + 'voted_lab_top100'  # 新标签文件夹路径

# 调用函数
copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder)

/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/AMOS-833/amos_7749_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/voted_lab/amos_7749.nii.gz
Copied amos_7749.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/AMOS-833/amos_7933_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/voted_lab/amos_7933.nii.gz
Copied amos_7933.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/AMOS-833/amos_8099_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/AMOS-833/voted_lab/amos_8099.nii.gz
Copied amos_8099.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/AMOS

In [14]:

# copy selected voted LLD samples to new folder
base_dir = "/home/data3/whm/code/flare25/datasets/"
dataset='LLD-MMRI-3984'

image_folder = base_dir + f'FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/{dataset}/'  # 图像文件夹路径
label_folder = base_dir + f'pseudolab/1adj_spacing/train_MRI_unlabeled/{dataset}/voted_lab'  # 标签文件夹路径
txt_file = base_dir + f'pseudolab/1adj_spacing/train_MRI_unlabeled/{dataset}/output_cls_13_scores_topk.txt'   # 包含选中样本名的txt文件

new_folder_base = f"/home/data3/whm/code/flare25/datasets/voted_sample/{dataset}/"
new_image_folder = new_folder_base + 'imageTr_top100'  # 新图像文件夹路径
new_label_folder = new_folder_base + 'voted_lab_top100'  # 新标签文件夹路径

# 调用函数
copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder)

/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/MR-459946_5_OutPhase_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/voted_lab/MR-459946_5_OutPhase.nii.gz
Copied MR-459946_5_OutPhase.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/MR182036_0_OutPhase_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/voted_lab/MR182036_0_OutPhase.nii.gz
Copied MR182036_0_OutPhase.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/MR69723_5_OutPhase_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_MRI_unlabeled/LLD-MMRI-3984/voted_lab/MR69723_5_OutPhase.nii.gz
Copied MR69723_5_OutPhase.nii.gz to new f

In [15]:
# copy selected voted PET-13 samples to new folder
base_dir = "/home/data3/whm/code/flare25/datasets/"
dataset='train_PET_unlabeled'

image_folder = base_dir + f'FLARE-Task3-DomainAdaption/processed/1adj_spacing/{dataset}'  # 图像文件夹路径
label_folder = base_dir + f'pseudolab/1adj_spacing/{dataset}/voted_lab/o_13'  # 标签文件夹路径
txt_file = base_dir + f'pseudolab/1adj_spacing/{dataset}/CT_MR_TJU_nonli/output_cls_13_scores_topk.txt'   # 包含选中样本名的txt文件

new_folder_base = f"/home/data3/whm/code/flare25/datasets/voted_sample/{dataset}_13/"
new_image_folder = new_folder_base + 'imageTr_top100'  # 新图像文件夹路径
new_label_folder = new_folder_base + 'voted_lab_top100'  # 新标签文件夹路径

# 调用函数
copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder)

/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_PET_unlabeled/fdg_556b56485a_08-29-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-83019_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/voted_lab/o_13/fdg_556b56485a_08-29-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-83019.nii.gz
Copied fdg_556b56485a_08-29-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-83019.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_PET_unlabeled/fdg_bd7603e9df_05-06-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-26867_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/voted_lab/o_13/fdg_bd7603e9df_05-06-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-26867.nii.gz
Copied fdg_bd7603e9df_05-06-2005-NA-PET-CT_Ganzkoerper__primaer_mit_KM-26867.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed

In [16]:
# copy selected voted PET-4 samples to new folder
base_dir = "/home/data3/whm/code/flare25/datasets/"
dataset='train_PET_unlabeled'

image_folder = base_dir + f'FLARE-Task3-DomainAdaption/processed/1adj_spacing/{dataset}'  # 图像文件夹路径
label_folder = base_dir + f'pseudolab/1adj_spacing/{dataset}/voted_lab/o_4'  # 标签文件夹路径
txt_file = base_dir + f'pseudolab/1adj_spacing/{dataset}/output_cls_13_4_scores_topk.txt'   # 包含选中样本名的txt文件

new_folder_base = f"/home/data3/whm/code/flare25/datasets/voted_sample/{dataset}_4/"
new_image_folder = new_folder_base + 'imageTr_top100'  # 新图像文件夹路径
new_label_folder = new_folder_base + 'voted_lab_top100'  # 新标签文件夹路径

# 调用函数
copy_selected_samples(txt_file, image_folder, label_folder, new_image_folder, new_label_folder)

/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_PET_unlabeled/fdg_185da4c8b6_04-18-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-06919_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/voted_lab/o_4/fdg_185da4c8b6_04-18-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-06919.nii.gz
Copied fdg_185da4c8b6_04-18-2003-NA-PET-CT_Ganzkoerper__primaer_mit_KM-06919.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_PET_unlabeled/fdg_9f6e8b1b43_11-16-2002-NA-PET-CT_Ganzkoerper_nativ-64835_0000.nii.gz
/home/data3/whm/code/flare25/datasets/pseudolab/1adj_spacing/train_PET_unlabeled/voted_lab/o_4/fdg_9f6e8b1b43_11-16-2002-NA-PET-CT_Ganzkoerper_nativ-64835.nii.gz
Copied fdg_9f6e8b1b43_11-16-2002-NA-PET-CT_Ganzkoerper_nativ-64835.nii.gz to new folders.
/home/data3/whm/code/flare25/datasets/FLARE-Task3-DomainAdaption/processed/1adj_spacing/train_PET_unlabele